In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("unsloth/Qwen2.5-3B-Instruct-bnb-4bit", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("unsloth/Qwen2.5-3B-Instruct-bnb-4bit")

print(f"Model loaded successfully on {device}.")

In [2]:
import os

def generate_new_name(original_name, tokenizer, model, device="cuda"):
    prompt = (f"Try to identify the name of a song in the following input texts. Only return the song name:'{original_name}'. "
              "Output only the new name you guess without any additional text.")

    messages = [{"role": "user", "content": prompt}]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512, do_sample=True)

    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

In [ ]:
folder_path = "/mnt/d/网络下载音乐/"  # Replace with your folder path

# Iterate over files in the folder and rename them based on model output.
for filename in os.listdir(folder_path):
    original_path = os.path.join(folder_path, filename)

    # Process only files (skip directories)
    if os.path.isfile(original_path):
        print(f"Processing file: {filename}")

        # Generate a new file name using the model
        new_base_name = generate_new_name(filename, tokenizer, model, device)

        # Optionally, preserve the original extension if not included in the generated name.
        original_ext = os.path.splitext(filename)[1]
        if not new_base_name.endswith(original_ext):
            new_base_name += original_ext

        # Prepare the new file path
        new_file_path = os.path.join(folder_path, new_base_name)

        # If the file already exists, append a numeric suffix until it's unique.
        count = 1
        base_name, ext = os.path.splitext(new_base_name)
        while os.path.exists(new_file_path):
            new_base_name = f"{base_name}_{count}{ext}"
            new_file_path = os.path.join(folder_path, new_base_name)
            count += 1

        try:
            os.rename(original_path, new_file_path)
            print(f"Renamed '{filename}' to '{new_base_name}'\n")
        except Exception as e:
            print(f"Error renaming '{filename}': {e}\n")